In [54]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [55]:
# read in all the words
words = open('../../names.txt', 'r').read().splitlines()

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [56]:
# sum([ 1 for word in words if word.startswith('a')])

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [57]:
# for k,v in itos.items():
#     print(f"{v}-{sum(1 for word in words if word.startswith(v))}")

# words_count = {}
# for k,v in itos.items():
#     for word in words:
#         if word.startswith(v):
#             second_sequence = v + word[1]
#             words_count[second_sequence] = words_count.get(second_sequence, 0) + 1


# print(words_count)

# sorted(words_count.items(), key = lambda k:-k[1])

    

In [58]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [59]:
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

# import random
# random.seed(42)
# random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(["cab"])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [60]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
  p.requires_grad = True


C1 = C.clone()

In [61]:
lri = []
lossi = []
stepi = []

In [62]:
import numpy as np


initial_weights = [p.clone() for p in parameters]

# List to store weight changes for visualization
weight_changes = []

for i in range(20000):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32,))
  # forward pass
  emb = C[Xtr] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, 6) @ W1) # (32, 200)
  logits = h @ W2  # (32, 27)
  loss = F.cross_entropy(logits, Ytr)
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad


  # track stats
  #lri.append(lre[i])
  # stepi.append(i)
  # lossi.append(loss.log10().item())

In [63]:
%reload_ext tensorboard

In [64]:
# Assuming you've already loaded your Jupyter notebook in VSCode and are ready to go

# Import necessary libraries
import torch
from torch.utils.tensorboard import SummaryWriter

# Load TensorBoard magic command in Jupyter

# Create dummy embeddings and metadata for demonstration
# Replace this with your actual embeddings if you have them
#metadata = [f'Point {i}' for i in range(182625)]  # Just using a generic label here for demonstration
metadata = ["".join([itos[idx] for idx in seq]) for seq in Xtr.tolist()]

# Set up TensorBoard writer and add embeddings
writer = SummaryWriter('runs/embeddings_demo_5')

# Register the embeddings and the metadata
writer.add_embedding(C[Xtr].view(-1,30), metadata=metadata, tag='my_30d_embeddings_5')

writer.close()

# Start TensorBoard within the notebook
%tensorboard --logdir runs/embeddings_demo_5


In [65]:
# Assuming you've already loaded your Jupyter notebook in VSCode and are ready to go

# Import necessary libraries
import torch
from torch.utils.tensorboard import SummaryWriter

# Load TensorBoard magic command in Jupyter

# Create dummy embeddings and metadata for demonstration
# Replace this with your actual embeddings if you have them
#metadata = [f'Point {i}' for i in range(182625)]  # Just using a generic label here for demonstration
metadata = ["".join([itos[idx] for idx in seq]) for seq in Xtr.tolist()]

# Set up TensorBoard writer and add embeddings
writer = SummaryWriter('runs/embeddings_demo_4')

# Register the embeddings and the metadata
writer.add_embedding(C1[Xtr].view(-1,30), metadata=metadata, tag='my_30d_embeddings_4')

writer.close()

# Start TensorBoard within the notebook
%tensorboard --logdir runs/embeddings_demo_4


Reusing TensorBoard on port 6006 (pid 12272), started 0:23:48 ago. (Use '!kill 12272' to kill it.)

In [66]:
Xtr


tensor([[ 0,  0,  0],
        [ 0,  0, 25],
        [ 0, 25, 21],
        ...,
        [15, 12,  4],
        [12,  4,  1],
        [ 4,  1, 14]])

In [67]:
str(stoi)


"{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}"

In [68]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.3760, grad_fn=<NllLossBackward0>)

In [69]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.3827, grad_fn=<NllLossBackward0>)

In [70]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

... ---> m
..m ---> o
.mo ---> b
mob ---> a
oba ---> c
bac ---> i
aci ---> m
cim ---> y
imy ---> a
mya ---> h
yah ---> .
mobacimyah.
... ---> s
..s ---> e
.se ---> e
see ---> .
see.
... ---> m
..m ---> e
.me ---> d
med ---> .
med.
... ---> l
..l ---> y
.ly ---> l
lyl ---> a
yla ---> .
lyla.
... ---> l
..l ---> e
.le ---> v
lev ---> e
eve ---> r
ver ---> .
lever.
... ---> j
..j ---> e
.je ---> n
jen ---> d
end ---> r
ndr ---> a
dra ---> r
rar ---> i
ari ---> .
jendrari.
... ---> l
..l ---> e
.le ---> e
lee ---> .
lee.
... ---> e
..e ---> d
.ed ---> i
edi ---> e
die ---> l
iel ---> i
eli ---> e
lie ---> l
iel ---> l
ell ---> y
lly ---> y
lyy ---> .
edieliellyy.
... ---> j
..j ---> e
.je ---> n
jen ---> .
jen.
... ---> e
..e ---> m
.em ---> e
eme ---> i
mei ---> g
eig ---> h
igh ---> a
gha ---> n
han ---> a
ana ---> n
nan ---> a
ana ---> r
nar ---> a
ara ---> e
rae ---> l
ael ---> y
ely ---> n
lyn ---> i
yni ---> m
nim ---> a
ima ---> l
mal ---> q
alq ---> u
lqu ---> a
qua ---> .
emeighan